# 📌 Résumé

## 🔍 Modèles utilisés

1. **Régression logistique**  
   Modèle de base simple et rapide à entraîner. Très utile comme point de comparaison.  
   - ✅ Interprétable : les coefficients indiquent l’influence de chaque variable. (utile dans un contexte metier)
   - ✅ Performant sur des données linéairement séparables.
   - ❌ Peut être limité si les relations entre variables sont non linéaires.

2. **Arbre de décision**  
   Méthode intuitive qui segmente l’espace de décision via des règles simples.  
   - ✅ Facilement lisible et interprétable.
   - ✅ Capacité à identifier les variables les plus importantes.
   - ❌ Risque de surapprentissage s’il n’est pas bien régularisé.

3. **Random Forest**  
   Ensemble d’arbres de décision créés à partir de sous-échantillons aléatoires des données.  
   - ✅ Réduit le surapprentissage grâce à l’agrégation (bagging).
   - ✅ Généralement plus robuste et performant qu’un arbre unique.
   - ✅ Moins coûteux en calcul et plus facile à paramétrer qu’un modèle comme XGBoost.
   - ❌ Moins interprétable qu’un arbre simple, mais on peut extraire l’importance des variables.

---

## 🎯 Objectif d'optimisation : le **F1-score**

L'objectif est de prédire quels clients vont **résilier**.
Nous avons choisi d'optimiser le **F1-Score**

- **Précision** = parmi les clients que le modèle prédit comme "à risque", combien le sont réellement ?
- **Rappel** = parmi les clients qui vont réellement résilier, combien le modèle arrive-t-il à détecter ?

Optimiser uniquement la précision :
- ➕ Moins de faux positifs.
- ➖ Mais risque de rater beaucoup de vrais résiliants.

Optimiser uniquement le rappel :
- ➕ On identifie la majorité des vrais résiliants.
- ➖ Mais on cible aussi beaucoup de clients à tort (faux positifs), ce qui coûte cher.

👉 **F1-score = compromis entre précision et rappel**  
C’est une métrique équilibrée, idéale dans notre cas où :
- Les classes sont déséquilibrées ainsi l'accuracy n'est pas réellement interpretable.
- Les **faux négatifs** (clients à risque non détectés) et les **faux positifs** (clients mal classés) ont **tous deux un impact business important**.

---

## ⚙️ Optimisation des hyperparamètres

Pour améliorer les performances des modèles, plusieurs méthodes ont été utilisées :

1. **Recherche par grille (Grid Search)**  
   - Explore toutes les combinaisons possibles de paramètres dans un espace défini.

2. **Recherche aléatoire (Random Search)**  
   - Tire au hasard des combinaisons dans l’espace des paramètres. Plus rapide que Grid Search dans de nombreux cas.

3. **Optimisation bayésienne**  
   - Utilise les résultats précédents pour choisir plus intelligemment les combinaisons suivantes.
   - Plus efficace et économe en calcul, surtout avec des modèles coûteux à entraîner.

